<a href="https://colab.research.google.com/github/thnavear/Qualcomm-DL-Hackathon/blob/master/genai_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 5.2 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/thnavear/Qualcomm-DL-Hackathon.git

In [2]:
from groq import Groq

client = Groq(api_key="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")
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": "Hello"}], # Added a user message
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


#os.environ["TOGETHER_API_KEY"]="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"

Hello. How can I assist you today?

In [9]:
# prompt: 1.code to import "/content/test.csv"
# 2.use model "llama-3.3-70b-versatile" from groq to classify emails from test.csv (read columns "ticket_ID", "ticket_subject" and "ticket_body" from each row in "/content/test.csv" into the following attributes
# Department: The responsible department, such as Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry
# Priority: The urgency level such as low, medium, or high.
# Language: The language code for the email's language, such as en (English), de (German) etc. You can access the list of language codes from the following link.
# Type: The nature of the request, categorized as Incident, Request, Problem, or Change
# Return your response as a JSON object with the following structure:
#    {
#        "Department": "...",
#        "Priority": "...",
#        "Language": "...",
#        "Type": "..."
#    }
# Convert the JSON object into output.csv with columns "ticket_ID","department","type","priority","language"

import json
import pandas as pd

# Process results and create a list of dictionaries
processed_results = []
for index, result in enumerate(results):
    try:
        json_data = json.loads(result)
        processed_results.append({
            "ticket_ID": df.iloc[index]['ticket_ID'],  # Assuming 'ticket_ID' exists
            "department": json_data.get("Department", ""),
            "type": json_data.get("Type", ""),
            "priority": json_data.get("Priority", ""),
            "language": json_data.get("Language", "")
        })
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for result {index + 1}: {e}")
        print(f"Problematic Result: {result}")  # Print problematic result for debugging
        processed_results.append({
            "ticket_ID": df.iloc[index]['ticket_ID'],
            "department": "",
            "type": "",
            "priority": "",
            "language": ""
        })


# Create a DataFrame from the processed results
output_df = pd.DataFrame(processed_results)

# Save the DataFrame to a CSV file
output_df.to_csv("output.csv", index=False)

NameError: name 'results' is not defined

In [17]:
# prompt: 1.code to import "/content/test.csv"
# 2.use model "llama-3.3-70b-versatile" from groq to classify emails from test.csv (read columns "ticket_ID", "ticket_subject" and "ticket_body" from each row in "/content/test.csv" into the following attributes
# Department: The responsible department, such as Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry
# Priority: The urgency level such as low, medium, or high.
# Language: The language code for the email's language, such as en (English), de (German) etc. You can access the list of language codes from the following link.
# Type: The nature of the request, categorized as Incident, Request, Problem, or Change
# 3.Return your response as a JSON object with the following structure:
#    {
#        "Department": "...",
#        "Priority": "...",
#        "Language": "...",
#        "Type": "..."
#    }
# 4.Convert the JSON object into output.csv with columns "ticket_ID","department","type","priority","language"

import pandas as pd
import json
from groq import Groq

# Install necessary libraries
!pip install groq

# Initialize the Groq client
client = Groq(api_key="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")

try:
    df = pd.read_csv("/content/test1.csv") # Correct file name
except FileNotFoundError:
    print("Error: test.csv not found. Please upload the file to /content/")
    exit()

results = []
for index, row in df.iterrows():
    ticket_id = row['ticket_ID']
    subject = row['ticket_subject']
    body = row['ticket_body']

    prompt = f"""Classify the following email into the given attributes.

Subject: {subject}
Body: {body}

Attributes:
Department: The responsible department (Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry)
Priority: The urgency level (low, medium, or high).
Language: The language code (en, de, etc.).
Type: The nature of the request (Incident, Request, Problem, or Change)

Return your response as a JSON object with the keys "Department", "Priority", "Language", and "Type"."""

    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_completion_tokens=200,
        top_p=1,
        stream=False
    )
    response = completion.choices[0].message.content
    #response_str = response.strip()
    response_str = response.replace("```","")
    response_str = response_str.replace("json","")
    response_str = response_str.replace("\n","")
    results.append(response_str)

processed_results = []
for index, result in enumerate(results):
    try:
        json_data = json.loads(result)
        processed_results.append({
            "ticket_ID": df.iloc[index]['ticket_ID'],
            "department": json_data.get("Department", ""),
            "type": json_data.get("Type", ""),
            "priority": json_data.get("Priority", ""),
            "language": json_data.get("Language", "")
        })
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for result {index + 1}: {e}")
        print(f"Problematic Result: {result}")
        processed_results.append({
            "ticket_ID": df.iloc[index]['ticket_ID'],
            "department": "",
            "type": "",
            "priority": "",
            "language": ""
        })

output_df = pd.DataFrame(processed_results)
output_df.to_csv("output.csv", index=False)